In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import time
import pymongo

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.mars_db
collection = db.items

In [4]:
#Headlines

In [5]:
url = 'https://mars.nasa.gov/news/'
response = requests.get(url)

headlines_soup = bs(response.text, 'lxml')

headline = headlines_soup.find('div', class_ = 'content_title').text.strip()

print(headline)

NASA Garners 7 Webby Award Nominations


In [6]:
news = headlines_soup.find('div', class_ = 'rollover_description_inner').text.strip()

print(news)

Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.


In [7]:
#Image of Mars

In [8]:
from selenium import webdriver
from splinter import Browser
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [9]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [10]:
browser.click_link_by_id('full_image')


In [11]:
time.sleep(2)

In [12]:
html = browser.html
image_soup = bs(html, "lxml")
img_url = image_soup.find('img', class_ = 'fancybox-image')['src']

In [13]:
featured_img_url = "https://www.jpl.nasa.gov" + img_url

featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18816_ip.jpg'

In [14]:
#Twitter 

In [15]:
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)

In [16]:
weather_soup = bs(response.text, 'html.parser')


In [17]:
mars_weather_tweet = weather_soup.find_all('div', class_ = "js-tweet-text-container")


In [18]:
for tweet in mars_weather_tweet:
    if tweet.text.strip().startswith('InSight'):
        mars_weather = tweet.text.strip()

In [19]:
print(mars_weather)

InSight sol 117 (2019-03-26) low -107.3ºC (-161.1ºF) high -17.1ºC (1.2ºF)
pressure at 7.30 hPa


In [20]:
# Mars Facts

In [21]:
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [22]:
mars_df = pd.read_html(url)
mars_df = (mars_df[0])

In [23]:
mars_df.columns = ["Description", "Value"]

mars_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [24]:
mars_df = mars_df.to_html(classes='mars')
table_data = mars_df.replace('\n', ' ')
table_data

'<table border="1" class="dataframe mars">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Description</th>       <th>Value</th>     </tr>   </thead>   <tbody>     <tr>       <th>0</th>       <td>Equatorial Diameter:</td>       <td>6,792 km</td>     </tr>     <tr>       <th>1</th>       <td>Polar Diameter:</td>       <td>6,752 km</td>     </tr>     <tr>       <th>2</th>       <td>Mass:</td>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>3</th>       <td>Moons:</td>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <th>4</th>       <td>Orbit Distance:</td>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <th>5</th>       <td>Orbit Period:</td>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <th>6</th>       <td>Surface Temperature:</td>       <td>-153 to 20 °C</td>     </tr>     <tr>       <th>7</th>       <td>First Record:</td>       <td>2nd millennium BC</td>     </tr>     <tr>       <th>8</t

In [25]:
#Database

In [26]:
post = {
    'headline': headline, 
    'news': news, 
    'featured': featured_img_url,
    'weather': mars_weather,
    'facts': table_data
}
collection.insert_one(post)

In [27]:

webdatas = db.items.find()

for webdata in webdatas:
    print(webdata)

{'_id': ObjectId('5cb5324ff403d4371cac9741'), 'headline': 'NASA Garners 7 Webby Award Nominations', 'news': 'Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.', 'featured': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA20465_ip.jpg', 'weather': 'InSight sol 117 (2019-03-26) low -107.3ºC (-161.1ºF) high -17.1ºC (1.2ºF)\npressure at 7.30 hPa', 'facts': '<table border="1" class="dataframe mars">   <thead>     <tr style="text-align: right;">       <th></th>       <th>Description</th>       <th>Value</th>     </tr>   </thead>   <tbody>     <tr>       <th>0</th>       <td>Equatorial Diameter:</td>       <td>6,792 km</td>     </tr>     <tr>       <th>1</th>       <td>Polar Diameter:</td>       <td>6,752 km</td>     </tr>     <tr>       <th>2</th>       <td>Mass:</td>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <th>3</th>       <td>Moons:</td>